### Extract from OSM
The purpose of this script is to show how to go from a .osm.pbf file to a network format that GOSTnets will accept. 
Additional modifications in this script include: clipping the roads to an input polygon (here, kinshasa.shp), and slicing the roads DataFrame by a list of accepted road types, as tagged in OSM (see accepted_road_types list). 

Before running this, make sure to register your sparkly new environment. By typing: 
conda activate myenv 

python -m ipykernel install --user --name [py36]  \

Note that after installing GOSTnets as follows: \

conda create --name test python=3.6 \
conda activate test \
conda install -c conda-forge rtree geopandas rasterio geojson \
git clone https://github.com/worldbank/GOSTnets.git \
python setup.py build \
python setup.py install \
\
Then you manually download: \
\
conda install networkx \
pip install osmnx \
conda install numpy


In [7]:
import sys
#!conda install --yes --prefix {sys.prefix} boltons
import geopandas as gpd
import pandas as pd
import os, sys
import gdal

# add to your system path the location of the LoadOSM.py and GOSTnet.py scripts
codepath = r"C:\Users\wb495141\Documents\GitHub\GOSTnets"

sys.path.append(codepath)
import GOSTnets as gn
from GOSTnets import load_osm as losm
import importlib

Define filepaths. Since we just need some randome baseline OSM data to which we can add a new road shapefile, we use uzbekistan-latest.osm.pbf. But this could be anything.

In [13]:
pth = r'C:/Users/wb495141/OneDrive - WBG/poverty/chad/input/road/'
pth2 = r'C:/Users/wb495141/OneDrive - WBG/poverty/chad/temp/'
fil = r'uzbekistan-latest.osm.pbf'
f = os.path.join(pth, fil)
f

'C:/Users/wb495141/OneDrive - WBG/poverty/chad/input/road/uzbekistan-latest.osm.pbf'

Using the loadOSM library, pass the filepath of the .osm.pbf to initiate a OSM_to_network class

In [14]:
road_osm = losm.OSM_to_network(f)

C:\WBG\Anaconda3\envs\py36\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


This generates a property of the class called 'roads_raw'. This is a raw dataframe of the highway objects, extracted from the .osm.pbf. This is the starting point for our network

In [15]:
actual_chad = gpd.read_file(os.path.join(pth, 'road.shp'))

In [16]:
actual_chad['infra_type'] = 'road'
actual_chad['osm_id'] = actual_chad.index
actual_chad = actual_chad[['osm_id','infra_type','geometry','LENGTH']]
actual_chad

,osm_id,infra_type,geometry,LENGTH
0,0,road,"LINESTRING (18.33049 21.74411, 18.34925 21.783...",60.760
1,1,road,"LINESTRING (18.02387 21.72066, 18.04944 21.724...",33.049
2,2,road,"LINESTRING (19.21814 21.56853, 19.23689 21.591...",39.488
3,3,road,"LINESTRING (19.21716 21.56672, 19.21741 21.567...",0.225
4,4,road,"LINESTRING (19.30348 21.49296, 19.37666 21.525...",41.913
...,...,...,...,...
2221,2221,road,"LINESTRING (15.82716 7.63409, 15.82126 7.64952...",26.909
2222,2222,road,"LINESTRING (16.99294 7.84913, 16.97882 7.82449...",29.065
2223,2223,road,"LINESTRING (15.67789 7.67464, 15.67968 7.67237...",18.219
2224,2224,road,"LINESTRING (16.10453 7.72293, 16.10565 7.69377...",14.875


In [17]:
road_osm.roads_raw = actual_chad

In [18]:
road_osm.roads_raw.infra_type.value_counts()

road    2226
Name: infra_type, dtype: int64

Note that kin.generateRoadsGDF runs only if input roadshapefile is projected in WGS84

In [19]:
road_osm.generateRoadsGDF(verbose = False)

C:\WBG\Anaconda3\envs\py36\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


With initialReadIn(), we transform this to a graph object

In [20]:
road_osm.initialReadIn()

We save this graph object down to file using gn.save()

In [21]:
gn.save(road_osm.network,'chad_G_unclean',pth2)